In [ ]:
using BasisFunctions, FrameFun, Plots, DomainSets, LinearAlgebra
gr()

# Basis platforms

A platform is a generator of dictionaries. Each dictionary corresponds to a parameter value, that is typically an integer. For example:

In [ ]:
plat = FourierPlatform()
dictionary(plat, 10)

### The Fun interface

Platforms are convenient to compute approximations with. The platform corresponds to a sequence of dictionaries, each corresponding to a parameter value (or a set of parameter values). Typically, the sequence is dense in a function space.

You can specify the parameter when invoking Fun, and compute an approximation in the corresponding dictionary. Alternatively, you can only specify the platform in which case an adaptive approximation is constructed from the sequence of dictionaries in the platform.

First, we choose the parameter `n=10` to approximate a periodic function using Fourier series:

In [ ]:
f = x-> 1/(2+cos(2*pi*x))^2
F = Fun(f, plat, 10)

We manually evaluate the error by computing the average residual in a set of random points:

In [ ]:
Q = 50
x = rand(Q)
sum(abs.(F.(x)-f.(x)))/Q

In [ ]:
plot(F)
plot!(f)

What is the best value of n? We leave away the parameter and try to find out with an adaptive approximation.

In [ ]:
F = Fun(f, plat, tol=1e-14)

In [ ]:
sum(abs.(F.(x)-f.(x)))/Q

Instead of a platform, we can also just specify a concrete dictionary to use. The approximation will be computed using default algorithms (that are nevertheless specific to this dictionary). The advantage of using a platform is that you have more control over which algorithms are used: more on that further down.

In [ ]:
F = Fun(f, ChebyshevT(40))

In [ ]:
F = Fun(f, Legendre(40))

Finally, if you want to compute adaptive approximations based on an example of a dictionary, you can easily make a `ModelPlatform`. This defines a simple platform based on resizing the given dictionary.

In [ ]:
F = Fun(f, platform(ChebyshevT(2)))

### Verbosity is informative

What happened under the hood in these simple examples? We mentioned that the code uses "default algorithms" and that they depend on the dictionary or platform supplied. Which algorithms? It is informative to specify the verbose option:

In [ ]:
F = Fun(f, plat, 40, verbose=true)

Several items are displayed here:
- the *dictionary* that is used for the approximation: in this case a Fourier series on the default interval `0..1`
- the *sampling style*: in the case above `InterpolationStyle` is used. This means that the function `f` is evaluated in the interpolation grid that is associated with the dictionary.
- the *solver style*: the approximation problem leads to a linear system `Ax=B`. The solver style dictates how this system is solved. In this case, `InverseStyle` is used, which means the system is solved as `x = inv(A)*B`. This is not a good idea in general. It is a good idea in this case because `A` corresponds to an FFT which is easily (and efficiently) invertible. It is the fastest way to solve this particular system.

We show the output of `verbose=true` when using adaptivity:

In [ ]:
F = Fun(f, plat, verbose=true)

The adaptive algorithm solves several approximation problems. It uses the verbose option only the first time: this shows which algorithms are used for a single approximation problem. Normally, the same algorithm is used in all subsequent solves. The example above uses `InterpolationStyle` for the sampling and `InverseStyle` for the solver.

The remaining output shows information about the convergence of the adaptive algorithm. Here, the adaptive style is reported to be `OptimalStyle`. This is an algorithm that computes the optimal length of the approximation by first repeatedly doubling `n` until the approximation tolerance is achieved. Next, in phase 2, the optimal value of `n` is determined by bisection.

### Changing which algorithm is used via optional arguments
All choices can be changed by specifying other arguments.

For example, you can specify to use a direct solver to solve `Ax=B`, even though the system is solved most efficiently by an FFT. The default direct solver is currently based on an SVD decomposition. (Of course you would not normally do this.)

In [ ]:
F = Fun(f, plat, 10, solverstyle = DirectStyle(), verbose=true)

Note that the residual that is being mentioned is the residual of the system `Ax=B` that is solved. In this case of interpolation, the system can be solved with small error. Yet, small residual here does not imply a small approximation error! It only means that the interpolation problem can be solved to high accuracy.

The `solverstyle` argument only specifies the algorithm used in the solve step. Additional arguments for the algorithms are passed separately. For example, you can specify to use QR instead of SVD.

In [ ]:
F = Fun(f, plat, 10, solverstyle = DirectStyle(), directsolver = :qr, verbose=true)

Similarly, you can change the sampling style by specifying the `samplingstyle` argument. The `GramStyle` correponds to a continuous approximation problem in which the matrix `A` is the Gram matrix. 

This is typically a bit slower, because the continuous projection requires the evaluation of inner products. There are efficient ways of constructing the Gram matrix for several dictionaries. However, the entries of the right hand side in `Ax=B` involver integrals with `f`, which are evaluated numerically.

In [ ]:
F = Fun(f, plat, 10, samplingstyle = GramStyle(), verbose=true)

The solver is chosen to be a direct solver. However, since the matrix `A` is diagonal (the Fourier series is orthogonal after all), the direct solver just corresponds to inverting this matrix.

You can change the `adaptivestyle` as well. The `SimpleStyle` algorithm simply doubles `n` until convergence.

In [ ]:
F = Fun(f, plat, adaptivestyle = SimpleStyle(), verbose=true)

Finally, all options can be combined.

In [ ]:
F = Fun(f, plat, solverstyle = DirectStyle(), directsolver = :qr, adaptivestyle = SimpleStyle(), verbose=true)

### Inspecting the discretization and algorithms

The `FrameFun` package focuses on flexibility in choosing discretizations and algorithms. For convenience, each step of an approximation problem can be inspected separately, with an interface that is much like the Fun interface itself.

That is, you can either specify a platform and parameter value, or a specific dictionary.

Each approximation problem corresponds to the solution of a linear system `Ax=B`. The problem is discretized using a sampling operator `S`, governed by the sampling style. The samling operator applied to the function `f` yields the right hand side `B`. When applied to the dictionary itself, it yields `A`.

In [ ]:
S = samplingoperator(plat, 100)

We can construct a Fun, and then reconstruct the approximation manually to convince ourselves that we know what is going on.

In [ ]:
F = Fun(f, plat, 100)

In [ ]:
S = samplingoperator(plat, 100)
basis = dictionary(plat, 100)
A = S * basis
B = S * f
c = matrix(A) \ B
norm(c -  coefficients(F))

Of course you don't need to do this by hand. The `discretization` routine returns precisely the linear system that is constructed as part of solving an approximation problem.

In [ ]:
A,B = discretization(f, plat, 100)
A

In [ ]:
size(B)

If you only care about the matrix `A`, leave out the function.

In [ ]:
A = discretization(plat, 100)

A side remark. In case of Fourier series, an efficient evaluation using the FFT is available on periodic grids.
The inverse of this operator is again realized using an FFT.

In [ ]:
inv(A)

Now, how about changing some options?

If the sampling style is different, the sampling operator changes accordingly.

In [ ]:
samplingoperator(plat, 10, samplingstyle=GramStyle())

Making a basis on the interval `2..5` simply results in a mapped periodic grid. The evaluation matrix `A` is identical to the one above.

In [ ]:
samplingoperator(Fourier(100) → 2..5)

In [ ]:
discretization(Fourier(100) → 2..5)

# Platforms for frames

It is straightforward to implement a special-purpose platform. However, some default ones are provided.

The simplest one is the univariate Fourier extension platform. We extend a function on  `0..0.5` to a periodic function on `0..1`.

In [ ]:
P = FourierExtensionPlatform(0.0..0.5)

In [ ]:
F = Fun(exp, P, 100)

In [ ]:
plot(F)

You can ask which dictionary a platform parameter corresponds to. In this case, it will be an extension frame with support on `0..0.5`:

In [ ]:
dict = dictionary(P, 10)

In [ ]:
support(dict)

An extension frame knows that it is a restriction of a basis on a larger domain:

In [ ]:
superdict(dict)

In [ ]:
support(superdict(dict))

#### Dimensions of the operators

An efficient discrete Fourier extension frame approximation has three relevant parameters: `N`, `M` and `L`.
- `N` is the length of the Fourier series
- `M` is the number of points in the oversampled grid
- `L` is less visible but often relevant as well. In a Fourier extension from `0..0.5` to  `0..1` we use `M` equispaced points on `0..0.5`. These points are a subset of an equispaced grid with `L` points on `0..1`.

We first compute the sampling operator of our Fourier extension frame, which by default uses an oversampled (least squares) grid.

In [ ]:
S = samplingoperator(P, 10)

Here, `N=10` is given as the argument in the function call. The grid of this operator (an `IndexSubGrid`) has length `M=20`. It is visible from the output that this grid is realized as the subset of a larger grid (a `FourierGrid` on `0...1`) of length `L=38`.

The size of the operator is `(20,Inf)` because it acts on an infinite-dimensional space (a function space).

In [ ]:
size(S)

By specifying an `M` parameter in addition to `N` you can control the size of the oversampled grid.

In [ ]:
S = samplingoperator(P, 10, M=120)
size(S)

Unfortunately, it is not always straightforward to guess which size `L` of the larger grid yields precisely size `M` of the subgrid. This, in fact, may require quite some computation. Thus, constructing the sampling grid when `M` is given is hard. In some cases, no value of `L` will produce exactly the `M` that was requested. In that case, the smallest value of `L` is deduced for which the length of the grid exceeds `M`.

On the other hand, computing the grid when `L` is given is easy and it is a viable option too.

In [ ]:
samplingoperator(P, 10, M=120, verbose = true)

In [ ]:
samplingoperator(P, 10, L=239, verbose = true)

#### Structure of the discretization matrix

The discretization matrix `A` in case of an extension frame has a very special structure. It consists of an extension of the Fourier series (of length `N`) to a bigger Fourier series (of length `L`), followed by an FFT, followed by a restriction to the equispaced grid (of length `M`) of the smaller interval `0..0.5`:

In [ ]:
A = discretization(P, 10)

The middle FFT operator acts on the larger Fourier series, in this case of length `L=38`. It has a periodic grid on `0..1` with 38 points, such that the restriction of that grid to `0..0.5` results in an oversampled grid for the frame. Implementing the matrix vector product for the frame in terms of this larger FFT is a crucial step in achieving efficiency of the solver.

You can query the source (`src`) and destination (`dest`) of each operator. This way you can confirm that there is a larger FFT basis with an associated grid.

In [ ]:
src(element(A,2))

In [ ]:
dest(element(A,2))

#### Weighted sum frames

Here is a fancier frame, a so-called weighted sum frame. Its dictionaries are weighted sums of dictionaries.

You can create a weighted sum platform from an existing platform as follows.

In [ ]:
WP = WeightedSumPlatform(P, x->sqrt(x),x->1.)

The dictionaries of this platform are now the dictionaries of the underlying platform (Fourier extensions on `0..0.5`), weighted by the functions `x->1` and `x->sqrt(x)` respectively:

In [ ]:
dictionary(WP, (10,10))

In [ ]:
F = Fun( x-> sqrt(sin(x)) + cos(x), WP)

The discretization matrix of this approximation problem now is considerably more involved, but it has suitable structure to enable fast matrix-vector products and the fast construction and application of a dual matrix Z as well (more on that in the AZ algorithm section below).

In [ ]:
discretization(WP, (10,10))

## The AZ algorithm

The AZ algorithm solves the system `Ax=B` in a sequence of three steps:
1. (I - AZ')A x1 = (I-AZ')B
2. x2 = Z' * (B-A*x1)
3. x = x1+x2

The factor `I-AZ'` is called the *plunge operator*.

The `A` matrix of the AZ algorithm is of course just the matrix of the system `Ax=B`.
For the Fourier extension platform we created above, it is:

In [ ]:
AZ_A(P, 101)

In this case, the `Z` matrix is fairly similar, differing just by a scalar factor.

In [ ]:
AZ_Z(P, 101)

The plunge operator corresponds precisely to `(I-AZ')` with `A` and `Z` as above. It is a little more complicated, yet it is clear that a fast matrix-vector product is available.

In [ ]:
plungeoperator(P, 100)

If your are familiar with the theory of prolate spheroidal wave functions, notice how the `E`, `R` and `F` operators above act together to form timelimiting and bandlimiting operators.
The plunge operator acts on the time domain. We have:
- `E` is the extension of the subgrid on `0..0.5` to the larger periodic grid on `0..1` of length `L` (time domain)
- the product `F1 * F2` is the projection of a Fourier series of length `L` to one of length `N`, followed by the extension back to length `L` using zero padding. This is a bandlimiting operation in the frequency domain.
- `R` is the restriction back to the subgrid, in the time domain.

The plunge operator of the weighted sum platform is more daunting to look at, but upon closer inspection it is a block-matrix generalization of the operator shown above. Most importantly, it can be generated automatically and it can be applied efficiently.

In [ ]:
plungeoperator(WP, (100,100))

## Creating custom platforms

Let's say I am dissatisfied with the ChebyshevPlatform because I want to use different points. Rather than the default interpolation grid of roots of Chebyshev polynomials (nodes):

In [ ]:
typeof(interpolation_grid(ChebyshevT(10)))

...I want to use the extremae:

In [ ]:
typeof(BasisFunctions.transformgrid_extremae(ChebyshevT(10)))

In [ ]:
BasisFunctions.transformgrid_extremae(ChebyshevT(5))

Here is how I might go about doing that:

In [ ]:
struct MyChebyshevPlatform <: FrameFun.BasisPlatform
end

FrameFun.correctparamformat(::MyChebyshevPlatform,::Int) = true

BasisFunctions.dictionary(p::MyChebyshevPlatform, n) = ChebyshevT(n)

FrameFun.interpolation_grid(p::MyChebyshevPlatform, n; dict, options...) = BasisFunctions.transformgrid_extremae(dict)

In [ ]:
samplingoperator(ChebyshevPlatform(), 10)

In [ ]:
samplingoperator(MyChebyshevPlatform(), 10)

The roots of Chebyshev polynomials are associated with a DCT of type III:

In [ ]:
discretization(ChebyshevPlatform(), 100)

The extremae correspond to DCTII instead. The type you see below is an efficient plan produced by the FFTW library for the DCT-II type transform.

In [ ]:
discretization(MyChebyshevPlatform(), 100)